In [1]:
import sys
import tensorflow as tf
sys.path.append('../../')
from dvalib import detector

Using TensorFlow backend.


In [2]:
text_detector = detector.TextBoxDetector(model_path='../../repos/tf_ctpn_cpu/checkpoints/')

In [3]:
text_detector.load()

Tensor("Placeholder:0", shape=(?, ?, ?, 3), dtype=float32)
Tensor("conv5_3/conv5_3:0", shape=(?, ?, ?, 512), dtype=float32)
Tensor("rpn_conv/3x3/rpn_conv/3x3:0", shape=(?, ?, ?, 512), dtype=float32)


Tensor("lstm_o/Reshape:0", shape=(?, ?, ?, 128), dtype=float32)
Tensor("lstm_o/Reshape:0", shape=(?, ?, ?, 128), dtype=float32)
Tensor("rpn_cls_score/Reshape:0", shape=(?, ?, ?, 20), dtype=float32)
Tensor("rpn_cls_prob:0", shape=(?, ?, ?, ?), dtype=float32)
Tensor("Reshape_5:0", shape=(?, ?, ?, 20), dtype=float32)
Tensor("rpn_bbox_pred/Reshape:0", shape=(?, ?, ?, 40), dtype=float32)
Tensor("Placeholder_1:0", shape=(?, 3), dtype=float32)
INFO:tensorflow:Restoring parameters from ../../repos/tf_ctpn_cpu/checkpoints/VGGnet_fast_rcnn_iter_50000.ckpt


INFO:tensorflow:Restoring parameters from ../../repos/tf_ctpn_cpu/checkpoints/VGGnet_fast_rcnn_iter_50000.ckpt


In [5]:
from glob import glob
try:
    os.mkdir('boxes')
except:
    pass
box_count = 0
for im_name in glob("images/*.jpg"):
    boxes = text_detector.detect(im_name)
    break

In [ ]:
from glob import glob
timer=Timer()
try:
    os.mkdir('boxes')
except:
    pass
box_count = 0
for im_name in glob("images/*.jpg"):
    im_file=osp.join(im_name)
    im=cv2.imread(im_file)
    timer.tic()
    im, f=resize_im(im, cfg.SCALE, cfg.MAX_SCALE)
    text_lines=text_detector.detect(im)        
    im_with_text_lines=draw_boxes(im, text_lines, caption=im_name, is_display=False)
    cv2.imwrite('out.jpg',im_with_text_lines)
    display(Image('out.jpg'))
    for k in text_lines:
        top,left,bottom,right,score = k 
        crop_img = im[int(left):int(right),int(top):int(bottom)]
        cv2.imwrite('boxes/box_{}.jpg'.format(box_count),crop_img)
        display(Image('boxes/box_{}.jpg'.format(box_count)))
        box_count += 1